## Setup

### Packages

In [1]:
import numpy as np
from numpy.polynomial import Polynomial as pol
from matplotlib import pyplot as plt
import matplotlib as mpl
import copy
import itertools
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from scipy.optimize import minimize

### Own modules

In [2]:
import import_ipynb
from COSMETICS import *

importing Jupyter notebook from COSMETICS.ipynb


### Cosmetics

In [3]:
plt.rc('font',**{'family':'sans-serif', 'sans-serif':['Arial'], 'size':14})
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['xtick.major.size'] = 7
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['axes.xmargin'] = 0.03
mpl.rcParams['axes.ymargin'] = 0.03
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.constrained_layout.use'] = True

### Ocean component bifurcation

In [4]:
def f(e1, e2, e3):
    '''Returns equilibria (T, S) of the Stommel box model for a choice of Stommel parameters
    e1 = eta_1, e2 = eta_2, and e3 = eta_3. Here, we use defining (REF) equation (1.12) for Psi'''
    
    # equilibria when Psi is positive
    # positive equilibrium values of Psi must satisfy a cubic with the following coefficients: 
    poscoeffs = [e2-e1*e3, e2-e1+e3, 1+e3, 1]
    # retrieve roots of polynomial with coefficients poscoeffs:
    posrts = (pol(poscoeffs)).roots()
    # only keep positive real roots:
    posrts = [rt for rt in posrts if abs(rt.imag)<1e-6 and rt>0]
    # determine associated S and T values:
    ST = [(e2/(e3+np.abs(rt)), e1/(1+np.abs(rt))) for rt in posrts]
    
    # equilibria when Psi is negative (similar to above)
    negcoeffs = [e2-e1*e3, e1-e2+e3, -1-e3, 1]
    negrts = (pol(negcoeffs)).roots()
    negrts = [rt for rt in negrts if abs(rt.imag)<1e-6 and rt<0]
    
    # add S and T equilibrium values associated with negative Psi
    # to list of all S and T equilibria:
    for rt in negrts:
        ST.append((e2/(e3+np.abs(rt)), e1/(1+np.abs(rt))))

    return ST

def J(S, T, e3):
    '''Returns the eigenvalues of the Jacobian of the Stommel box model in the point (S, T)
    for a given value of e3 = eta_3 (note that the Jacobian of the Stommel box model does not depend
    on eta_1 and eta_2); also see (REF) Section 1.2.'''
    # Recall that the Jacobian depends on the sign of T-S:
    sgn = np.sign(T-S)
    jacobian = np.matrix([[-1+sgn*(S-2*T), sgn*T],[-sgn*S, -e3+sgn*(2*S-T)]])
    return list(np.linalg.eigvals(jacobian))

def type_of_eq(S, T, e3, lst=True):
    '''Obtain type of equilibrium'''
    (l1, l2) = J(S, T, e3)
    if abs(l1.imag) < 1e-5:
        if l1*l2 < 0:
            return('S')
        else:
            if (l1>0 and l2 > 0):
                return('uN')
            else:
                return('sN')
        
    else:
        if l1.real > 0:
            return('uF')
        else:
            return('sF')

### Ice component

In [5]:
# The sea ice component

def seaice(params, t, dt, I0, sI, R):
    
    [D, R0, B] = params
    
    nt = int(t/dt)
    time = np.arange(0, t, dt)
    W_I = np.sqrt(dt)*np.random.randn(nt)
    I = np.zeros(nt)
    I[0] = I0
    
    for i in range(nt-1):
        I[i+1] = I[i] + dt*(D*np.tanh(I[i]) + (R0*np.heaviside(I[i], 0)-B)*I[i] + R) + sI*W_I[i]
        
    return time, I

### Ice bifurcation using pseudo-arc continuation

In [6]:
# Pseudo-arc continuation; p. 266 in https://ipsen.math.ncsu.edu/ps/sinum65438.pdf

D = 0.86
R_0 = -0.1
B = 0.45

def f1(I, R):
    return D * np.tanh(I) + (R_0*np.heaviside(I, 0) - B)*I + R

def df1I(I, R):
    return D * 1/(np.cosh(I))**2 + R_0*np.heaviside(I, 0)-B

def df1R(I, R):
    return 1

def f2(I, I_s, I_g, R, R_s, R_g, ds): 
    return (I_g-I)*I_s + (R_g-R)*R_s-ds 

def newton(I, I_s, I_g, R, R_s, R_g, ds):
    correction = [1, 1]
    while abs(correction[0]**2 + correction[1]**2) > 1e-3:
        Jac = np.array([[df1I(I_g, R_g), 1], [I_s, R_s]])
        Jacinv = np.linalg.inv(Jac)
        correction = np.dot(Jacinv, np.array([f1(I_g, R_g), f2(I, I_s, I_g, R, R_s, R_g, ds)]))
        I_g = I_g - correction[0]
        R_g = R_g - correction[1]
    return [I_g, R_g]

def psarc(I_init, R_init, s_end, ds): # e.g. -1.2, -0.35, 1.1, 0.01  
    # pseudo-arc continuation
    s = 0
    R = R_init 
    I = I_init
    L = []
    f_R, f_I = 1, df1I(I, R)
    R_s = np.sqrt(1/(1+f_I**(-2)))
    I_s = -1/f_I * R_s
    while s < s_end:
        # determine initial guesses
        R_g = R + R_s * ds
        I_g = I + I_s * ds
        # determine new I, R values with Newton method
        [I, R] = newton(I, I_s, I_g, R, R_s, R_g, ds)
        L.append((R, I))
        # determine next direction vector and normalize
        [I_s, R_s] = np.dot(np.linalg.inv(np.array([[df1I(I, R), df1R(I, R)], [I_s, R_s]])), np.array([0, 1]))
        [I_s, R_s] = [I_s, R_s]/np.linalg.norm([I_s, R_s])
        # update s
        s = s + ds
    return L

In [7]:
def findI(R, saddle=False):
    D = 0.86
    B = 0.45
    R0 = -0.1
    i1 = np.arccosh(np.sqrt(D/(B-R0)))
    lpl = D*np.tanh(i1) + (R0-B)*i1
    error = 1
    tolerance = 0.00001
    I_low = i1
    I_high = 5
    
    while error > tolerance:
        I = (I_low + I_high)/2
        difference = D * np.tanh(I) + (R_0*np.heaviside(I, 0) - B)*I + R
        error = abs(difference)
        if difference > 0:
            I_low = I
        else:
            I_high = I
                
    return I

In [8]:
def findsaddle(R, c, T):
    D = 0.86
    B = 0.45
    R0 = -0.1
    i1 = np.arccosh(np.sqrt(D/(B-R0)))
    i2 = -np.arccosh(np.sqrt(D/B))
    I_low = i2
    I_high = i1
    difference = 1
    tolerance = 0.00001
    
    while abs(difference) > tolerance:
        I = (I_low + I_high)/2
        difference = D * np.tanh(I) + (R_0*np.heaviside(I, 0) - B)*I + R + c*T
#         print(I, difference)
        if difference > 0:
            I_high = I
        else:
            I_low = I
                
    return I

In [9]:
e1 = 3.0
e2 = 1.0
e3 = 0.3

def findT(R, c, k):
    
    Tlow = 2
    Thigh = 3.333
    difference = 1
    tolerance = 0.000000001
    
    while abs(difference) > tolerance:
        T = (Tlow + Thigh)/2
        while e1-k*findI(R+c*T) < 2.55:
            T -= 0.01
        while e1-k*findI(R+c*T) > 3.333:
            T += 0.01
        for pt in f(e1-k*findI(R+c*T), e2, e3):
            if type_of_eq(pt[0], pt[1], e3) == 'S':
                difference = pt[1] - T
                if difference > 0:
                    Tlow = T
                else:
                    Thigh = T
                
    return T

## Slow system

In [11]:
# The slow system

D = 0.86
R0 = -0.1 
B = 0.45
i1 = np.arccosh(np.sqrt(D/(B-R0))) # bifurcation point ice system (ice-free branch)
i2 = -np.arccosh(np.sqrt(D/B)) # bifurcation point ice system (ice-covered branch)

def f0(I, R):
    # ice system
    return D * np.tanh(I) + (R0*np.heaviside(I, 0)-B)*I + R

def IinTprep(R, dx, c, Tmax=6, Imin=-3, Imax=3):
    # numerically invert T = -1/c f0(I, R) to obtain a function I=I(T)
    # (upper and lower branch separately). Tmax is the maximal value of
    # T that we want to be able to plug into I(T), and Imin and Imax are
    # the minimal and maximal values that I attains. Smaller dx means
    # the inverse is calculated at more points
    
    def diff(x, a):
        return (a + 1/c*f0(x, R))**2
    
    arr = -1/c*f0(np.arange(i1, Imax, dx), R)
    T_u = np.arange(max(np.min(arr), 0), min(np.max(arr), Tmax), dx) # T-coordinate axis
    I_u = np.zeros(T_u.shape) # This will be the I value corresponding to each T value in T_u

    for idx, x_value in enumerate(T_u): # invert T = -1/c f0(I, R)
        res = minimize(diff, i1, args=(x_value), method='Nelder-Mead', tol=1e-6)
        I_u[idx] = res.x[0]

    arr = -1/c*f0(np.arange(Imin, i2, dx), R)
    T_l = np.arange(max(0, np.min(arr)), min(np.max(arr), Tmax), dx)
    I_l = np.zeros(T_l.shape)

    for idx, x_value in enumerate(T_l):
        res = minimize(diff, i2, args=(x_value), method='Nelder-Mead', tol=1e-6)
        I_l[idx] = res.x[0]
    
    return T_u, T_l, I_u, I_l

def IinT(T, I_branch, R, dx, c, T_u, T_l, I_u, I_l):
    # finds I_+(T) if I_term > 0 (i.e. 'upper') 
    # or I_-(T) if I_term <= 0 (i.e. 'lower')
    
    if len(T_u) != 0 and len(T_l) != 0:
        if I_branch > 0:
            # system was on the upper branch before
            if T >= T_u[0]:
                # system is still on the upper branch
                pre_ind = (T-T_u[0])/dx
                ind = int(pre_ind)
                if ind <= len(I_u)-2:
                    I_term = (1-(pre_ind-ind))*I_u[ind] + (pre_ind-ind)*I_u[ind+1]
                else:
                    I_term = I_u[ind]
            else:
                # system now tips to the lower branch
                pre_ind = (T-T_l[0])/dx
                ind = int(pre_ind)
                if ind <= len(I_l)-2:
                    I_term = (1-(pre_ind-ind))*I_l[ind] + (pre_ind-ind)*I_l[ind+1]
                else:
                    I_term = I_l[ind]
        else:
            # system was on the lower branch before
            if T >= T_l[-1]:
                # system now tips to the upper branch
                pre_ind = (T-T_u[0])/dx
                ind = int(pre_ind)
                if ind <= len(I_u)-2:
                    I_term = (1-(pre_ind-ind))*I_u[ind] + (pre_ind-ind)*I_u[ind+1]
                else:
                    I_term = I_u[ind]
            else:
                # system is still on the lower branch
                pre_ind = (T-T_l[0])/dx
                ind = int(pre_ind)
                if ind <= len(I_l)-2:
                    I_term = (1-(pre_ind-ind))*I_l[ind] + (pre_ind-ind)*I_l[ind+1]
                else:
                    I_term = I_l[ind]
                
        return I_term
    
    elif len(T_u) == 0 and len(T_l) != 0:
        if T <= T_l[-1] and T >= T_l[0]:
            pre_ind = (T-T_l[0])/dx
            ind = int(pre_ind)
            if ind <= len(I_l)-2:
                I_term = (1-(pre_ind-ind))*I_l[ind] + (pre_ind-ind)*I_l[ind+1]
            else:
                I_term = I_l[ind]
            return I_term
        else:
            return('FLAG0')
        
    elif len(T_u) != 0 and len(T_l) == 0:
        if T >= T_u[0] and T <= T_u[-1]:
            pre_ind = (T-T_u[0])/dx
            ind = int(pre_ind)
            if ind <= len(I_u)-2:
                I_term = (1-(pre_ind-ind))*I_u[ind] + (pre_ind-ind)*I_u[ind+1]
            else:
                I_term = I_u[ind]
            return I_term
        else:
            return('FLAG1')
            
    else:
        return('FLAG2')


def TSslow(t, dt, T0, S0, e1, e2, e3, k, c, R, dx, ice, I_branch=1):
    '''Choosing I_branch = 1, resp. -1 means that the system is assumed to lie on
    the ice-covered, resp. ice-free branch at t=0.'''
    
    nt = int(t/dt)
    time = np.arange(0, t, dt)
    
    T = np.zeros(nt)
    S = np.zeros(nt)
    I = np.zeros(nt)
    T[0] = T0
    S[0] = S0
    
    [T_u, T_l, I_u, I_l] = ice
    I_term = I_branch
    I[0] = IinT(T[0], I_term, R, dx, c, T_u, T_l, I_u, I_l)
    
    for i in range(nt-1):
        T[i+1] = T[i] + dt * (e1 - k*np.heaviside(I[i],0)*I[i] - T[i]*(1+abs(T[i]-S[i])))
        S[i+1] = S[i] + dt * (e2-S[i]*(e3+abs(T[i]-S[i])))
        I[i+1] = IinT(T[i+1], I[i], R, dx, c, T_u, T_l, I_u, I_l)
    
    return time, I, T, S


# def TSslow_old(t, dt, T0, S0, e1, e2, e3, k, c, R, dx, I_branch=1):
#     '''Choosing I_branch = 1, resp. -1 means that the system is assumed to lie on
#     the ice-covered, resp. ice-free branch at t=0.'''
    
#     nt = int(t/dt)
#     time = np.arange(0, t, dt)
    
#     T = np.zeros(nt)
#     S = np.zeros(nt)
#     I = np.zeros(nt)
#     T[0] = T0
#     S[0] = S0
    
#     T_u, T_l, I_u, I_l = IinTprep(R, dx, c, Tmax = 8, Imax = 8, Imin = -8)
#     I_term = I_branch # choose any value at least as big i1 to start on the upper branch
#     I[0] = IinT(T[0], I_term, R, dx, c, T_u, T_l, I_u, I_l)
    
#     for i in range(nt-1):
#         T[i+1] = T[i] + dt * (e1 - k*np.heaviside(I[i],0)*I[i] -T[i]*(1+abs(T[i]-S[i])))
#         S[i+1] = S[i] + dt * (e2-S[i]*(e3+abs(T[i]-S[i])))
#         I[i+1] = IinT(T[i+1], I[i], R, dx, c, T_u, T_l, I_u, I_l)
    
#     return time, I, T, S, T_u, T_l, I_u, I_l

In [12]:
# The coupled deterministic sea ice-ocean model

def ITS(iceparams, Rparams, init, etavals, t, dt, tau, c):

    [D, R0, B, k] = iceparams
    [Rrampstart, Rramplength, Rstartval, Rendval] = Rparams
    [T0, S0, I0] = init
    [e1hat, e2, e3] = etavals
    
    # Discretisation & Initialisation
    nt = int(t/dt)
    time = np.arange(0, t, dt)
    
    T = np.zeros(nt)
    S = np.zeros(nt)
    I = np.zeros(nt)
    T[0] = T0
    S[0] = S0
    I[0] = I0

    e1list = [e1hat-k*I0*np.heaviside(I0, 0)]*nt
    # Ramping
    Rlength = int(Rramplength/dt)
    Rstart = int(Rrampstart/dt)
    Rend = Rstart+Rlength
    R = [Rstartval]*nt
    R[Rstart:Rend] = [Rstartval+(i-Rstart)/(Rend-Rstart)*(Rendval-Rstartval) for i in range(Rstart, Rend)]
    R[Rend:] = [Rendval]*(nt-Rend)
    
    for i in range(nt-1):
        I[i+1] = I[i] + dt*(D*np.tanh(I[i]) + (R0*np.heaviside(I[i], 0)-B)*I[i] +c*T[i] + R[i])# or F-0.2*T[i]
        T[i+1] = T[i] + dt*tau*(e1hat-k*I[i]*np.heaviside(I[i], 0)-T[i]-abs(T[i]-S[i])*T[i])
        S[i+1] = S[i] + dt*tau*(e2-e3*S[i]-abs(T[i]-S[i])*S[i])
        e1list[i+1] = e1hat - k*I[i]*np.heaviside(I[i], 0)
    
    return time, I, T, S, R, e1list

In [13]:
def ITeTpS(iceparams, Rparams, init, Tetavals, t, dt, tau, c, Tea, xi):
    
    [D, R0, B, k] = iceparams
    [Rrampstart, Rramplength, Rstartval, Rendval] = Rparams
    [Te0, Tp0, S0, I0] = init
    [Tpastart, Tpaend, e2, e3] = Tetavals
    
    # Discretisation & Initialisation
    nt = int(t/dt)
    time = np.arange(0, t, dt)
    
    Te = np.zeros(nt)
    Tp = np.zeros(nt)
    S = np.zeros(nt)
    I = np.zeros(nt)
    Te[0] = Te0
    Tp[0] = Tp0
    S[0] = S0
    I[0] = I0

    Tpalist = [Tea-Tpastart]*nt
    # Ramping
    Rlength = int(Rramplength/dt)
    Rstart = int(Rrampstart/dt)
    Rend = Rstart+Rlength
    R = [Rstartval]*nt
    R[Rstart:Rend] = [Rstartval+(i-Rstart)/(Rend-Rstart)*(Rendval-Rstartval) for i in range(Rstart, Rend)]
    R[Rend:] = [Rendval]*(nt-Rend)
    
    for i in range(nt-1):
        I[i+1]  = I[i]  + dt* (D*np.tanh(I[i]) + (R0*np.heaviside(I[i], 0)-B)*I[i] +c*Tp[i]+ R[i])  # or F-0.2*T[i]
        Te[i+1] = Te[i] + dt*tau* (Tea-Te[i] + (xi-1) *(Te[i]-Tp[i])*abs((Te[i]-Tp[i]-S[i]))) 
        Tp[i+1] = Tp[i] + dt*tau* (Tpaend+k*I[i]*np.heaviside(I[i], 0)-Tp[i]+xi*abs(Te[i]-Tp[i]-S[i])*(Te[i]-Tp[i]))
        S[i+1]  = S[i]  + dt*tau* (e2-e3*S[i]-abs(Te[i]-Tp[i]-S[i])*S[i])
        Tpalist[i+1] = Tea - Tpaend - k*I[i]*np.heaviside(I[i], 0)
    return time, I, Te, Tp, S, R, Tpalist